In [26]:
import re
import pandas as pd; import polars as pl
from lyricsgenius import Genius

ACCESS_TOKEN = r'UYxNJIrszBTYr4DCDuFZCenrTrCDLiYwpj0Z3Xdn4vNK98BWhhc5GWAj0NwbUjxb-'
genius = Genius(ACCESS_TOKEN)

def clean_string(string):
    regex = re.compile('[^a-zA-Z \s ,]')
    string = regex.sub('', string)

    return string

In [131]:
# Pull lyrics using track and artist from the Kaggle dataset

tracks = pl.read_csv('data/spotify-2023.csv', encoding='latin-1', columns=[0,1])
tracks = tracks.to_numpy()

lyrics = []
failed = []
for track in tracks[len(lyrics) + len(failed):]:
    for attempt in range(3):
        try:
            song = genius.search_song(track[0], artist=track[1])
            break
        except:
            pass
    if song:
        lyrics.append({
            'id': song.id,
            'title': song.title,
            'full_title': song.full_title,
            'title_with_featured': song.title_with_featured,
            'artist': song.artist,
            'lyrics_state': song.lyrics_state,
            'lyrics': song.lyrics,
            'csv_title': track[0],
            'csv_artist': track[1]
        })
    else:
        failed.append({'title': track[0], 'artist': track[1]})

Searching for "Counting Stars" by OneRepublic...
Done.
Searching for "Ghost" by Justin Bieber...
Done.
Searching for "Under The Influence" by Chris Brown...
Done.
Searching for "PRC" by Natanael Cano, Peso Pluma...
Done.
Searching for "Gasolina" by Daddy Yankee...
Done.
Searching for "One Dance" by Drake, WizKid, Kyla...
Done.
Searching for "Enchanted" by Taylor Swift...
Done.
Searching for "Save Your Tears" by The Weeknd...
Done.
Searching for "Sure Thing" by Miguel...
Done.
Searching for "Every Breath You Take - Remastered 2003" by The Police...
Done.
Searching for "The Night We Met" by Lord Huron...
Done.
Searching for "We Found Love" by Rihanna, Calvin Harris...
Done.
Searching for "When I Was Your Man" by Bruno Mars...
Done.
Searching for "Let Me Down Slowly" by Alec Benjamin...
Done.
Searching for "Am I Dreaming (Metro Boomin & A$AP Rocky, Roisee)" by A$AP Rocky, Metro Boomin, Roisee...
Done.
Searching for "Do I Wanna Know?" by Arctic Monkeys...
Done.
Searching for "Demons" by Im

The snippet below was used to re-run failed API calls using "cleaned" input arguments

In [210]:
new = []
f2 = []
for i in failed[1:]:
    track = clean_string(i['title'])
    artist = clean_string(i['artist'])
    song = genius.search_song(track, artist=artist)
    if song:
        new.append({
            'id': song.id,
            'title': song.title,
            'full_title': song.full_title,
            'title_with_featured': song.title_with_featured,
            'artist': song.artist,
            'lyrics_state': song.lyrics_state,
            'lyrics': song.lyrics,
            'csv_title': i['title'],
            'csv_artist': i['artist']
        })
        failed.remove(i)
    else:
        f2.append(i)

Searching for "Until I Found You with Em Beihold  Em Beihold Version" by Em Beihold, Stephen Sanchez...
No results found for: 'Until I Found You with Em Beihold  Em Beihold Version Em Beihold, Stephen Sanchez'
Searching for "Novidade na " by Mc Livinho, DJ Matt D...
No results found for: 'Novidade na  Mc Livinho, DJ Matt D'
Searching for "MONTAGEM  FR PUNK" by Ayparia, unxbected...
No results found for: 'MONTAGEM  FR PUNK Ayparia, unxbected'
Searching for "Tere Vaaste From Zara Hatke Zara Bachke" by SachinJigar, Shadab Faridi, Altamash Faridi, Amitabh Bhattacharya, Varun Jain...
No results found for: 'Tere Vaaste From Zara Hatke Zara Bachke SachinJigar, Shadab Faridi, Altamash Faridi, Amitabh Bhattacharya, Varun Jain'
Searching for "Novo Balan" by Veigh, Bvga Beatz, Supernova Ent, Prod Malax...
No results found for: 'Novo Balan Veigh, Bvga Beatz, Supernova Ent, Prod Malax'
Searching for "Gol Bolinha, Gol Quadrado " by Mc Pedrinho, DJ ...
No results found for: 'Gol Bolinha, Gol Quadrado

The snippet below uses some code modified from the Accumulator class used in the Spotify API notebook. The inputs herein come from the outputs of Spotify's Track API (collected in the Spotify API notebook).

In [29]:
tracks = pd.read_csv('tracks.csv')

successes = []
failures = []
output = []


for i, row in tracks.iterrows():
    track_name = row['name']
    artists = ', '.join(list(eval(row['artists']).values()))
    if i not in successes:
        max_retries = 5
        for attempt in range(max_retries):
            try:
                print(f'[{i}] Accessing endpoint with {track_name}, {artists}... (Attempt {attempt+1})')
                response = genius.search_song(track_name, artist=artists)

                o = [row['spotify_id'],response.title, response.artist, response.lyrics]
                output.append((o))
                successes.append(i)
                break
            except Exception as e:
                if attempt == max_retries-1:
                    failures.append([i, str(e)])

pd.DataFrame(output, columns=['sid', 'g_title', 'g_artist', 'lyrics']).to_csv('lyrics.csv', index=False)

In [65]:
# View the failed calls, dump ids into a file for later investigation
fid = [i[0] for i in failures]
import json
with open('failures.json', 'w') as f:
    f.write(json.dumps(fid))

tracks.iloc[fid].drop_duplicates()

,spotify_id,name,artists,popularity
252,1Y3LN4zO1Edc2EluIoSPJN,Until I Found You (with Em Beihold) - Em Beiho...,"{'5XKFrudbV4IiuE5WuTPRmT': 'Stephen Sanchez', ...",89
283,0p0cOpBujR114Wirv5AM7W,Narcotics (with Bryant Myers),"{'2YRyPiW98bpkARAS4B3OQP': 'Dei V', '6w9ToX5sl...",83
361,2tRcF1bE5ESw9c8ljLXLsR,Blowfly (with Ab-Soul),"{'28ExwzUQsvgJooOI0X1mr3': 'Jay Rock', '0g9vAl...",54
386,1qLBZLeGS6FEHgVKWqhuVv,007 (Jersey Mix),{'6I3MElirhT5t6Kf7p0hGk9': 'Duckwrth'},51
452,1TfqLAPs4K3s2rJMoCokcS,Sweet Dreams (Are Made of This) - Remastered,"{'0NKDgy9j66h3DLnN8qu1bB': 'Eurythmics', '5Msp...",85
...,...,...,...,...
1272,4nyMoK6gePLzCJybQl4O2F,Mtg - Coração,"{'6U0dJxYVB41L8WDZ02Nwuk': 'Dj Samir', '7i7ddO...",67
1273,2X1cIpDWE6nyBCde1iUBVM,Mtg Ensurdecência Estelar 1.0,"{'2YXKTfoEjKVzCNI0J6rHbQ': 'FR3ST', '0f1IECbrV...",71
1274,6393p1eg8oyVrB4z5jck1L,TOMA FORÇA CARIOCA,"{'6jisxxsMSH9OExkRD3YkwG': 'Hussain Ali', '1DB...",57
1275,658QJ4p8WcbT3O7iWLpO1D,Scary Garry,"{'3Tv1UzQAvn0v0hKTxisXnR': 'Kaito Shoma', '2le...",72
